<a href="https://colab.research.google.com/github/ViennaRNA/RNAdeep/blob/main/notebooks/spotrna_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount Drive
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
os.listdir()

In [ ]:
# Install ViennaRNA Package (needed for preprocessing)
# Get an old Miniconda that comes with Python 3.6.0 since Colab uses 3.6.9 in the webinterface
!wget https://repo.anaconda.com/miniconda/Miniconda3-4.3.11-Linux-x86_64.sh
!bash Miniconda3-4.3.11-Linux-x86_64.sh -b -f -p /usr/local/
!rm Miniconda3-4.3.11-Linux-x86_64.sh
!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge

!conda install -y  viennarna
!pip install numpy #not sure why this is not available anymore?

In [ ]:
#check import of RNA package
import sys
sys.path.append("/usr/local/lib/python3.6/site-packages") 
import RNA

In [ ]:
#copy data
!cp -r drive/My\ Drive/data/interim/10000_length70 .
!cp -r /content/drive/My\ Drive/data/normal_lowspike_25-100 .  #=dataset 4
!cp -r /content/drive/My\ Drive/data/normal_25-100 . #=dataset 3
!cp -r /content/drive/My\ Drive/data/uniform_25-100 . #=dataset 1
!cp -r /content/drive/My\ Drive/data/uniform_plus_25-100 . #=dataset 2

In [ ]:
#imports
!cp drive/My\ Drive/models/*py . #copy python files
from spotrna import spotrna
from metrics import mcc,f1,sensitivity
from data_generators_rna import PaddedDataMatrixFromInterimFile,DataMatrixFromInterimFile
import numpy as np
from tensorflow.keras.callbacks import CSVLogger,ModelCheckpoint
from tensorflow import keras
import os

In [ ]:
#Train with nonpadded data
#Model Settings
model = 1
batch_size = 16
num_epochs = 10

directory = "10000_length70"
train_generator = DataMatrixFromInterimFile(batch_size,os.path.join(directory,"train"))
val_generator = DataMatrixFromInterimFile(batch_size,os.path.join(directory,"val"))

m = spotrna(model,False)
#alernatively load model
#m = keras.models.load_model([path to saved model],custom_objects={"mcc":mcc,"f1":f1,"sensitivity":sensitivity})
csvname = "spotrna_%i_%i_%s.csv" % (batch_size,model,directory)
csv_logger = CSVLogger(csvname, append=True, separator=';')
m.compile(optimizer="adam",loss="binary_crossentropy", metrics=["acc",mcc,f1,sensitivity],run_eagerly=True)
checkpoint_filepath = "spotrna_%i_%i_%s" % (batch_size,model,directory)
#save model with best mcc
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_mcc', mode='max', save_best_only=True)
m.fit(x=train_generator, validation_data = val_generator,epochs=num_epochs,shuffle = True,verbose=1,callbacks=[csv_logger,model_checkpoint])
#save model after last epochs 
m.save("spotrna_%i_%i_%s_%i" % (batch_size,model,directory,epochs))

In [ ]:
#Train with nonpadded data
#Model Settings
model = 1
batch_size = 16
num_epochs = 10

directory = "uniform_25-100"
train_generator = PaddedDataMatrixFromInterimFile(batch_size,os.path.join(directory,"train_30k"))
val_generator = DataMatrixFromInterimFile(batch_size,os.path.join(directory,"val_5k"))

m = spotrna(model,True)
#alernatively load model
#m = keras.models.load_model([path to saved model],custom_objects={"mcc":mcc,"f1":f1,"sensitivity":sensitivity})
csvname = "spotrna_%i_%i_%s.csv" % (batch_size,model,directory)
csv_logger = CSVLogger(csvname, append=True, separator=';')
m.compile(optimizer="adam",loss="binary_crossentropy", metrics=["acc",mcc,f1,sensitivity],run_eagerly=True)
checkpoint_filepath = "spotrna_%i_%i_%s" % (batch_size,model,directory)
#save model with best mcc
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_mcc', mode='max', save_best_only=True)
m.fit(x=train_generator, validation_data = val_generator,epochs=num_epochs,shuffle = True,verbose=1,callbacks=[csv_logger,model_checkpoint])
#save model after last epochs 
m.save("spotrna_%i_%i_%s_%i" % (batch_size,model,directory,epochs))

In [ ]:
#use this to evaluate model
#[model].evaluate([generator])